In [25]:
import pandas as pd
import numpy as np

In [27]:
!pip install geocoder

In [29]:
import geocoder

Get the required data and work on it as before

In [31]:
url = "https://cocl.us/Geospatial_data"
loc_df = pd.read_csv(url)
loc_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [38]:
url2 = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url2,header=0)[0]
df = tables[~(tables['Borough'].eq('Not assigned') | tables['Borough'].eq('Not assigned'))]
grouped_final = df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(', '.join).reset_index()
grouped_final.columns = ['Postcode', 'Borough', 'Neighbourhood']
grouped_final

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [39]:
df_cd = pd.merge(grouped_final, loc_df,  how='left', left_on = 'Postcode', right_on = 'Postal Code')
df_cd

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...,...
98,M9N,York,Weston,M9N,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,43.739416,-79.588437


In [40]:
df_cd.drop('Postal Code', axis=1, inplace=True)
df_cd

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [41]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


We now have the data to work on

Get the geolocation of Toronto

In [42]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto is 43.65238435, -79.38356765.


In [43]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_cd['Latitude'], df_cd['Longitude'], df_cd['Borough'], df_cd['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [44]:
# The code was removed by Watson Studio for sharing.

In [45]:

LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 1000 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)



Get the neighborhood details

In [46]:
df_cd.loc[0, 'Neighbourhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [47]:
neighborhood_latitude = df_cd.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_cd.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_cd.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [48]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


Send a GET request for. the data

In [49]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60064b7718aab407238d1fbb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs'

Transorm the json file into a pandas data frame, but first create an empty data frame

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(


Take a look at the new dataframe

In [52]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


transform it into a dataframe

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,RBC Royal Bank,Bank,43.798782,-79.197090
4,Wendy’s,Fast Food Restaurant,43.807448,-79.199056




And how many venues were returned by Foursquare?

In [54]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


List the all the venues

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
toronto_venues = getNearbyVenues(names=df_cd['Neighbourhood'],
                                   latitudes=df_cd['Latitude'],
                                   longitudes=df_cd['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

Check the dataframe size

In [57]:
print(toronto_venues.shape)
toronto_venues.head()

(2105, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


How many unique categories are there?

In [58]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


##  Analyze Each Neighborhood

In [83]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehott = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
toronto_onehot.shape

(2105, 268)

In [84]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
toronto_grouped.shape

In [85]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1  Sandwich Place  0.14
2     Coffee Shop  0.14
3             Pub  0.14
4        Pharmacy  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0           Bank  0.10
1    Coffee Shop  0.10
2           Park  0.05
3  Deli / Bodega  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3            Boutique  0.05
4   Indian Rest

Put this into a dataframe

List the top 10 venues for each neighborhood

In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Sushi Restaurant,Café,Pub,Restaurant,Thai Restaurant


## Cluster Neighborhoods

In [88]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [89]:
toronto_grouped_clustering.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Sushi Restaurant,Café,Pub,Restaurant,Thai Restaurant


In [93]:
df_cd.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

In [94]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged = df_cd

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged2 = pd.merge(neighborhoods_venues_sorted,df_cd, on=["Neighborhood"])
neighborhoods_venues_sorted.insert(0, 'Cluster Labels1', kmeans.labels_)
toronto_merged2.head() # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket,M3H,North York,43.754328,-79.442259
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Sushi Restaurant,Café,Pub,Restaurant,Thai Restaurant,M5M,North York,43.733283,-79.419750


In [95]:
toronto_merged = df_cd

In [96]:
toronto_merged3 = pd.merge(neighborhoods_venues_sorted.set_index('Neighborhood'),df_cd, on=["Neighborhood"])


In [97]:
toronto_merged3.head() # check the last columns!

,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
0,Agincourt,3,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",3,Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",3,Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket,M3H,North York,43.754328,-79.442259
3,Bayview Village,3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",3,Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Sushi Restaurant,Café,Pub,Restaurant,Thai Restaurant,M5M,North York,43.733283,-79.419750


In [118]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged3['Latitude'], toronto_merged3['Longitude'], toronto_merged3['Neighborhood'], toronto_merged3['Cluster Labels1']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Exam the clusters 

Cluster 1

In [101]:
toronto_merged3

,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
0,Agincourt,3,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",3,Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",3,Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket,M3H,North York,43.754328,-79.442259
3,Bayview Village,3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M2K,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",3,Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Sushi Restaurant,Café,Pub,Restaurant,Thai Restaurant,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Willowdale East",3,Ramen Restaurant,Café,Sandwich Place,Pizza Place,Sushi Restaurant,Coffee Shop,Shopping Mall,Bubble Tea Shop,Discount Store,Lounge,M2N,North York,43.770120,-79.408493
95,"Willowdale, Willowdale West",3,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,M2R,North York,43.782736,-79.442259
96,Woburn,3,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,M1G,Scarborough,43.770992,-79.216917
97,Woodbine Heights,3,Skating Rink,Spa,Beer Store,Curling Ice,Dance Studio,Bus Stop,Park,General Travel,Golf Course,Dumpling Restaurant,M4C,East York,43.695344,-79.318389


List the venues in cluster 0

In [119]:
venues0 = toronto_merged3.loc[toronto_merged3['Cluster Labels1'] == 0, toronto_merged3.columns[[1] + list(range(0, toronto_merged3.shape[1]))]]
venues0

,Cluster Labels1,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
10,0,Caledonia-Fairbanks,0,Park,Bar,Pool,Women's Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,M6E,York,43.689026,-79.453512
30,0,"East Toronto, Broadview North (Old East York)",0,Park,Convenience Store,Intersection,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,M4J,East York,43.685347,-79.338106
34,0,"Forest Hill North & West, Forest Hill Road Park",0,Park,Bus Line,Sushi Restaurant,Jewelry Store,Trail,Yoga Studio,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,M5P,Central Toronto,43.696948,-79.411307
48,0,"Kingsview Village, St. Phillips, Martin Grove ...",0,Park,Sandwich Place,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,M9R,Etobicoke,43.688905,-79.554724
50,0,Lawrence Park,0,Business Service,Bus Line,Swim School,Park,Comfort Food Restaurant,Colombian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,M4N,Central Toronto,43.728020,-79.388790
54,0,"Milliken, Agincourt North, Steeles East, L'Amo...",0,Park,Intersection,Playground,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,M1V,Scarborough,43.815252,-79.284577
58,0,"North Park, Maple Leaf Park, Upwood Park",0,Park,Basketball Court,Bakery,Construction & Landscaping,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,M6L,North York,43.713756,-79.490074
65,0,Parkwoods,0,Park,Food & Drink Shop,Construction & Landscaping,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,M3A,North York,43.753259,-79.329656
69,0,Rosedale,0,Park,Playground,Trail,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,M4W,Downtown Toronto,43.679563,-79.377529
98,0,York Mills West,0,Convenience Store,Park,Electronics Store,Construction & Landscaping,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,M2P,North York,43.752758,-79.400049


Show the most popular venue on a map

In [137]:
# create map of Toronto venues in cluster 0 using latitude and longitude values
map_toronto_venues0 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
 
for indice, row in venues0.iterrows():
    folium.Marker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row['1st Most Common Venue'],
    icon=folium.map.Icon(color='orange')
    ).add_to(map_toronto_venues0)
    
map_toronto_venues0

List the venues in cluster 1

In [120]:
venues1 = toronto_merged3.loc[toronto_merged3['Cluster Labels1'] == 1, toronto_merged3.columns[[1] + list(range(0, toronto_merged3.shape[1]))]]
venues1

,Cluster Labels1,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
90,1,"West Deane Park, Princess Gardens, Martin Grov...",1,Bakery,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant,M9B,Etobicoke,43.650943,-79.554724


Show the most popular venue on a map

In [135]:
# create map of Toronto venues in cluster 1 using latitude and longitude values
map_toronto_venues1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
 
for indice, row in venues1.iterrows():
    folium.Marker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row['1st Most Common Venue'],
    icon=folium.map.Icon(color='orange')
    ).add_to(map_toronto_venues1)
    
map_toronto_venues1

List the venues in cluster 2

In [121]:
venues2 = toronto_merged3.loc[toronto_merged3['Cluster Labels1'] == 2, toronto_merged3.columns[[1] + list(range(0, toronto_merged3.shape[1]))]]
venues2

,Cluster Labels1,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
85,2,"The Kingsway, Montgomery Road, Old Mill North",2,River,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega,M8X,Etobicoke,43.653654,-79.506944


Show the most popular venue on the map

In [133]:
# create map of Toronto venues in cluster 2 using latitude and longitude values
map_toronto_venues2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
 
for indice, row in venues2.iterrows():
    folium.Marker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row['1st Most Common Venue'],
    icon=folium.map.Icon(color='orange')
    ).add_to(map_toronto_venues2)
    
map_toronto_venues2

Show the venues in cluster 3

In [122]:
venues3 = toronto_merged3.loc[toronto_merged3['Cluster Labels1'] == 3, toronto_merged3.columns[[1] + list(range(0, toronto_merged3.shape[1]))]]
venues3

,Cluster Labels1,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
0,3,Agincourt,3,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M1S,Scarborough,43.794200,-79.262029
1,3,"Alderwood, Long Branch",3,Pizza Place,Coffee Shop,Gym,Pharmacy,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,M8W,Etobicoke,43.602414,-79.543484
2,3,"Bathurst Manor, Wilson Heights, Downsview North",3,Coffee Shop,Bank,Park,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket,M3H,North York,43.754328,-79.442259
3,3,Bayview Village,3,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,M2K,North York,43.786947,-79.385975
4,3,"Bedford Park, Lawrence Manor East",3,Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store,Juice Bar,Sushi Restaurant,Café,Pub,Restaurant,Thai Restaurant,M5M,North York,43.733283,-79.419750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,3,"Wexford, Maryvale",3,Middle Eastern Restaurant,Smoke Shop,Auto Garage,Bakery,Shopping Mall,Vietnamese Restaurant,Sandwich Place,Yoga Studio,Distribution Center,Dim Sum Restaurant,M1R,Scarborough,43.750072,-79.295849
94,3,"Willowdale, Willowdale East",3,Ramen Restaurant,Café,Sandwich Place,Pizza Place,Sushi Restaurant,Coffee Shop,Shopping Mall,Bubble Tea Shop,Discount Store,Lounge,M2N,North York,43.770120,-79.408493
95,3,"Willowdale, Willowdale West",3,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,M2R,North York,43.782736,-79.442259
96,3,Woburn,3,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant,M1G,Scarborough,43.770992,-79.216917


Show the popular venues on a map

In [138]:
# create map of Toronto venues in cluster 1 using latitude and longitude values
map_toronto_venues3 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
 
for indice, row in venues1.iterrows():
    folium.Marker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row['1st Most Common Venue'],
    icon=folium.map.Icon(color='orange')
    ).add_to(map_toronto_venues3)
    
map_toronto_venues3

Show the venues in cluster 4

In [123]:
venues4 = toronto_merged3.loc[toronto_merged3['Cluster Labels1'] == 4, toronto_merged3.columns[[1] + list(range(0, toronto_merged3.shape[1]))]]
venues4

,Cluster Labels1,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude
53,4,"Malvern, Rouge",4,Fast Food Restaurant,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,M1B,Scarborough,43.806686,-79.194353


Show the popular venue on a map

In [139]:
# create map of Toronto venues in cluster 1 using latitude and longitude values
map_toronto_venues4 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
 
for indice, row in venues1.iterrows():
    folium.Marker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row['1st Most Common Venue'],
    icon=folium.map.Icon(color='orange')
    ).add_to(map_toronto_venues4)
    
map_toronto_venues4

Show the venues in cluster 5

In [124]:
venues5 = toronto_merged3.loc[toronto_merged3['Cluster Labels1'] == 5, toronto_merged3.columns[[1] + list(range(0, toronto_merged3.shape[1]))]]
venues5

,Cluster Labels1,Neighborhood,Cluster Labels1,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode,Borough,Latitude,Longitude


## According to FourSquare and the analysis there aren't many venues in the 5th cluster! no need to draw a map.